In [1]:
import spotipy
import pandas as pd
import numpy as np
from spotipy.oauth2 import SpotifyOAuth
import time

In [2]:
client_id='c4ff820f994745288df451ea7d6940d5'
client_secret='be2297719e5e4fb698f112c4374ee3b1'
redirect_uri='https://open.spotify.com/'
token_url = "https://accounts.spotify.com/api/token"

## Project Outline

- download a dataset of spotify playlists
- find or write a library to handle genre etc analysis
- group similar playlists
- use plotting techniques to show clusters?

Project Proposal:
I am personally interested in analyzing my longterm listening and curation history through Spotify and spotipy. I have an extensive collection of playlists, which due to their regularity, can be analyzed as objects. The intention of the analysis is to uncover trends in the groupings, visualize the relative distances (similarities) of songs within playlists, and create a prediction mechanism for most similar playlists. I will use sklearn, matplotlib, and seaborn for the clustering and visualizations.

## User Auth

In [83]:
scope = 'playlist-read-private'

sp_oauth = SpotifyOAuth(client_id=client_id,client_secret=client_secret,redirect_uri=redirect_uri,scope=scope)
token_info = sp_oauth.get_cached_token() 
if not token_info:
    auth_url = sp_oauth.get_authorize_url()
    print(auth_url)
    response = input('Paste the above link into your browser, then paste the redirect url here: ')

    code = sp_oauth.parse_response_code(response)
    token_info = sp_oauth.get_access_token(code)

    token = token_info['access_token']

sp = spotipy.Spotify(auth=token, retries=10, requests_timeout=10)

https://accounts.spotify.com/authorize?client_id=c4ff820f994745288df451ea7d6940d5&response_type=code&redirect_uri=https%3A%2F%2Fopen.spotify.com%2F&scope=playlist-read-private


/var/folders/dy/m7tb4mx52sv0lxfvtf95srbc0000gn/T/ipykernel_78143/4118829141.py:11: DeprecationWarning: You're using 'as_dict = True'.get_access_token will return the token string directly in future versions. Please adjust your code accordingly, or use get_cached_token instead.
  token_info = sp_oauth.get_access_token(code)


In [69]:
# scope = 'playlist-read-private'

# oauth = spotipy.util.prompt_for_user_token(client_id=client_id, client_secret=client_secret, redirect_uri=redirect_uri, scope=scope)
# sp = spotipy.Spotify(auth=oauth)

In [63]:
if sp:
    print(sp.me())

{'display_name': 'amaya lim', 'external_urls': {'spotify': 'https://open.spotify.com/user/nightrainlily'}, 'href': 'https://api.spotify.com/v1/users/nightrainlily', 'id': 'nightrainlily', 'images': [{'url': 'https://i.scdn.co/image/ab67757000003b82180bd1d0c567a1e26710aeb8', 'height': 64, 'width': 64}, {'url': 'https://i.scdn.co/image/ab6775700000ee85180bd1d0c567a1e26710aeb8', 'height': 300, 'width': 300}], 'type': 'user', 'uri': 'spotify:user:nightrainlily', 'followers': {'href': None, 'total': 355}}


## Getting Playlists

In [6]:
def get_all_playlists():
    playlist_count = 1000
    results = []
    offset = 0
    while offset < playlist_count:
        results.append(sp.current_user_playlists(limit=50, offset=offset))
        time.sleep(10)
        offset += 50
    return results
results = get_all_playlists()

In [25]:
def parse(playlist_json, target):
    lst = []
    for i in np.arange(len(playlist_json['items'])):
        target_value = playlist_json['items'][i]['track'][target]
        lst.append(target_value)
    return lst 

In [69]:
import re

playlist_data = []

for result in results:
    for playlist in result['items']:
        reg = "^([0-9])+\s([a-z]+(\s?)([a-z]?))"
        if re.search(reg, playlist['name']) != None:
            track_info = sp.playlist_tracks(playlist['id'], fields='items(track(id, name))')
            print(playlist)
            playlist_data.append({
                'name' : playlist['name'],
                'track_ids' : parse(track_info, 'id'),
                'track_names' : parse(track_info, 'name'),
                'id' : playlist['id'],
                'image' : playlist['images'][0]['url']
            })

playlists = pd.DataFrame(playlist_data)
playlists.head().style.format({'image': lambda x : f'<img src="{x}" width="100px"/>'})
# playlists.head()

{'collaborative': False, 'description': '', 'external_urls': {'spotify': 'https://open.spotify.com/playlist/4gmy8uL4ivI8J8FgZ2wcPi'}, 'href': 'https://api.spotify.com/v1/playlists/4gmy8uL4ivI8J8FgZ2wcPi', 'id': '4gmy8uL4ivI8J8FgZ2wcPi', 'images': [{'height': None, 'url': 'https://image-cdn-ak.spotifycdn.com/image/ab67706c0000da8461508206ba8c0dbb54e18d29', 'width': None}], 'name': '837 no pretending', 'owner': {'display_name': 'amaya lim', 'external_urls': {'spotify': 'https://open.spotify.com/user/nightrainlily'}, 'href': 'https://api.spotify.com/v1/users/nightrainlily', 'id': 'nightrainlily', 'type': 'user', 'uri': 'spotify:user:nightrainlily'}, 'primary_color': None, 'public': True, 'snapshot_id': 'AAAAIYSQBxvyxvu3WzvObocCTVefWh0d', 'tracks': {'href': 'https://api.spotify.com/v1/playlists/4gmy8uL4ivI8J8FgZ2wcPi/tracks', 'total': 10}, 'type': 'playlist', 'uri': 'spotify:playlist:4gmy8uL4ivI8J8FgZ2wcPi'}
{'collaborative': False, 'description': '', 'external_urls': {'spotify': 'https://

,name,track_ids,track_names,id,image
0,837 no pretending,"['2mWfVxEo4xZYDaz0v7hYrN', '1JVEM4C04VPAU9JTnKthNk', '5Qrcxc2YPomtX6UB0bwNaf', '397fxHew7NyjryGWGr1Imy', '7Kt7c9Q2B7hSyiTZvGQ37D', '6AcJTLIgNpxs5ERpo03QQc', '45JLSLXRMQJabYtPtSuBmV', '1jLNxsmGsEQAM4CUkHemfS', '5TOnnVV8YxB4NnRvavrPHX', '3qTJrHpLz7yY1SRux2Nf3H']","['Juna', '23', 'Love Is Stronger Than Pride', 'Baby, This Love I Have', 'Minetta Creek', 'Serenade of Water', 'Guess Who', '(There Is) No Greater Love', 'You the One Fish in the Sea', 'Oceanic Feeling']",4gmy8uL4ivI8J8FgZ2wcPi,
1,837 symmetrical,"['19RybK6XDbAVpcdxSbZL1o', '0p56A0wGIlQm0oKWeed6cm', '2KiQ23eB0oqyRQ7DoU744k', '5TOBZ7LCRmB7pddKAaCibB', '341bqzt1REzbvIodveSRSJ', '0rlX8acfJKPQ3NVHjxR7ml', '5LIUI808j8M8bXQd0kRxnp', '1yoqQTEmyvLC0uGbHTYqUU']","['Apple', 'Pitiful', 'cherry cola', 'tooth', 'She Loves Me', 'Icarus', 'Image', 'pretty when i cry']",3pK6VLkh6lwgjdkF8edW8t,
2,836 it falls apart,"['10cGyorRXoOc9RhVhY9Sru', '7hETG5A8HK2JIjnzUNhfK3', '2nes7xBxT8C2ol7lthypyF', '6NEnPyVS7opzxuT4f54V1y', '0nyJ9fGY32bcyxQo2BIMtK', '2HLvbuaEg93QWqip9AibIb']","[""She's Leaving You"", 'Right Before the Last Waves Took Vestris', 'Outfit', 'Tupelo', 'Desert', 'Open Water']",2rARwXlp3rvRwTDnT6KXXu,
3,836 i thought it was too late,"['3QWLuJZ2A5TJ1BN6H0AMg7', '43NNSXnxcclmlFuaTw2Lek']","['Vaughn', 'Mill']",1Do4dK0ls0jeeUTvx8yyDh,
4,836 coming around,"['6I9GeNnqNqQ5Q8DSrFkkvd', '6VNvyzWcgWUKsV7MkINTQQ', '0K69bmjZwfIXmlnFN43SgS', '2T9XG0dOWvJw23EUC8YIn5', '4Zg9es4eGGtFNUnnkCaOey', '1jlBePA3tcycCoANSSZbgj', '1RHVadqESon8w2Yv7QSsjH', '1Sbdw0lMZXkfVmjuH1UTb8', '0woEYg2SEO3u2MFKmVnDgM', '2M8VHnYfCGxS0DDA8n0yXX']","['Method Actor', 'Double', 'Midnight', 'AA BOUQUET FOR YOUR 180 FACE', 'Rylee & I', 'Hamptons', 'I Feel...', 'Thin Thing', 'Drum', 'Police Scanner']",5Iw1dfOuejmqp8b2YUYMHf,


In [78]:
def pretty_tracks(col, name):
        all_l = []
        for i in np.arange(len(playlists[col])):
                row = playlists[col].values[i]
                l = []
                for j in row:
                        for k in np.arange(len(row)):
                                track = item[k]['track']
                                id = track[name]
                                l.append(id)
                all_l.append(l)
        return all_l

# playlists['track_ids'] = pretty_tracks('track_ids', 'id')
# playlists['track_names'] = pretty_tracks('track_names', 'name')

# playlists.head().style.format({'image': lambda x : f'<img src="{x}" width="100px"/>'})


## Getting Song Data

In [75]:
songs = playlists
songs['zipp'] = list(zip(songs['track_ids'], songs['track_names']))
songs['zipp'][0]

(['2mWfVxEo4xZYDaz0v7hYrN',
  '1JVEM4C04VPAU9JTnKthNk',
  '5Qrcxc2YPomtX6UB0bwNaf',
  '397fxHew7NyjryGWGr1Imy',
  '7Kt7c9Q2B7hSyiTZvGQ37D',
  '6AcJTLIgNpxs5ERpo03QQc',
  '45JLSLXRMQJabYtPtSuBmV',
  '1jLNxsmGsEQAM4CUkHemfS',
  '5TOnnVV8YxB4NnRvavrPHX',
  '3qTJrHpLz7yY1SRux2Nf3H'],
 ['Juna',
  '23',
  'Love Is Stronger Than Pride',
  'Baby, This Love I Have',
  'Minetta Creek',
  'Serenade of Water',
  'Guess Who',
  '(There Is) No Greater Love',
  'You the One Fish in the Sea',
  'Oceanic Feeling'])

In [76]:
def merge(z):
    return list(tuple(zip(z[0], z[1])))

songs['zipp'] = songs['zipp'].apply(merge)
songs.head()

,name,track_ids,track_names,id,image,zipp
0,837 no pretending,"[2mWfVxEo4xZYDaz0v7hYrN, 1JVEM4C04VPAU9JTnKthN...","[Juna, 23, Love Is Stronger Than Pride, Baby, ...",4gmy8uL4ivI8J8FgZ2wcPi,https://image-cdn-ak.spotifycdn.com/image/ab67...,"[(2mWfVxEo4xZYDaz0v7hYrN, Juna), (1JVEM4C04VPA..."
1,837 symmetrical,"[19RybK6XDbAVpcdxSbZL1o, 0p56A0wGIlQm0oKWeed6c...","[Apple, Pitiful, cherry cola, tooth, She Loves...",3pK6VLkh6lwgjdkF8edW8t,https://mosaic.scdn.co/640/ab67616d00001e024d9...,"[(19RybK6XDbAVpcdxSbZL1o, Apple), (0p56A0wGIlQ..."
2,836 it falls apart,"[10cGyorRXoOc9RhVhY9Sru, 7hETG5A8HK2JIjnzUNhfK...","[She's Leaving You, Right Before the Last Wave...",2rARwXlp3rvRwTDnT6KXXu,https://mosaic.scdn.co/640/ab67616d00001e02012...,"[(10cGyorRXoOc9RhVhY9Sru, She's Leaving You), ..."
3,836 i thought it was too late,"[3QWLuJZ2A5TJ1BN6H0AMg7, 43NNSXnxcclmlFuaTw2Lek]","[Vaughn, Mill]",1Do4dK0ls0jeeUTvx8yyDh,https://i.scdn.co/image/ab67616d00001e0258c935...,"[(3QWLuJZ2A5TJ1BN6H0AMg7, Vaughn), (43NNSXnxcc..."
4,836 coming around,"[6I9GeNnqNqQ5Q8DSrFkkvd, 6VNvyzWcgWUKsV7MkINTQ...","[Method Actor, Double, Midnight, AA BOUQUET FO...",5Iw1dfOuejmqp8b2YUYMHf,https://image-cdn-ak.spotifycdn.com/image/ab67...,"[(6I9GeNnqNqQ5Q8DSrFkkvd, Method Actor), (6VNv..."


In [77]:
songs = songs.explode('zipp')
songs.head()

,name,track_ids,track_names,id,image,zipp
0,837 no pretending,"[2mWfVxEo4xZYDaz0v7hYrN, 1JVEM4C04VPAU9JTnKthN...","[Juna, 23, Love Is Stronger Than Pride, Baby, ...",4gmy8uL4ivI8J8FgZ2wcPi,https://image-cdn-ak.spotifycdn.com/image/ab67...,"(2mWfVxEo4xZYDaz0v7hYrN, Juna)"
0,837 no pretending,"[2mWfVxEo4xZYDaz0v7hYrN, 1JVEM4C04VPAU9JTnKthN...","[Juna, 23, Love Is Stronger Than Pride, Baby, ...",4gmy8uL4ivI8J8FgZ2wcPi,https://image-cdn-ak.spotifycdn.com/image/ab67...,"(1JVEM4C04VPAU9JTnKthNk, 23)"
0,837 no pretending,"[2mWfVxEo4xZYDaz0v7hYrN, 1JVEM4C04VPAU9JTnKthN...","[Juna, 23, Love Is Stronger Than Pride, Baby, ...",4gmy8uL4ivI8J8FgZ2wcPi,https://image-cdn-ak.spotifycdn.com/image/ab67...,"(5Qrcxc2YPomtX6UB0bwNaf, Love Is Stronger Than..."
0,837 no pretending,"[2mWfVxEo4xZYDaz0v7hYrN, 1JVEM4C04VPAU9JTnKthN...","[Juna, 23, Love Is Stronger Than Pride, Baby, ...",4gmy8uL4ivI8J8FgZ2wcPi,https://image-cdn-ak.spotifycdn.com/image/ab67...,"(397fxHew7NyjryGWGr1Imy, Baby, This Love I Have)"
0,837 no pretending,"[2mWfVxEo4xZYDaz0v7hYrN, 1JVEM4C04VPAU9JTnKthN...","[Juna, 23, Love Is Stronger Than Pride, Baby, ...",4gmy8uL4ivI8J8FgZ2wcPi,https://image-cdn-ak.spotifycdn.com/image/ab67...,"(7Kt7c9Q2B7hSyiTZvGQ37D, Minetta Creek)"


In [79]:
songs['track_name'] = songs['zipp'].map(lambda x: x[1])
songs['track_id'] = songs['zipp'].map(lambda x: x[0])
songs.head()

,name,track_ids,track_names,id,image,zipp,track_name,track_id
0,837 no pretending,"[2mWfVxEo4xZYDaz0v7hYrN, 1JVEM4C04VPAU9JTnKthN...","[Juna, 23, Love Is Stronger Than Pride, Baby, ...",4gmy8uL4ivI8J8FgZ2wcPi,https://image-cdn-ak.spotifycdn.com/image/ab67...,"(2mWfVxEo4xZYDaz0v7hYrN, Juna)",Juna,2mWfVxEo4xZYDaz0v7hYrN
0,837 no pretending,"[2mWfVxEo4xZYDaz0v7hYrN, 1JVEM4C04VPAU9JTnKthN...","[Juna, 23, Love Is Stronger Than Pride, Baby, ...",4gmy8uL4ivI8J8FgZ2wcPi,https://image-cdn-ak.spotifycdn.com/image/ab67...,"(1JVEM4C04VPAU9JTnKthNk, 23)",23,1JVEM4C04VPAU9JTnKthNk
0,837 no pretending,"[2mWfVxEo4xZYDaz0v7hYrN, 1JVEM4C04VPAU9JTnKthN...","[Juna, 23, Love Is Stronger Than Pride, Baby, ...",4gmy8uL4ivI8J8FgZ2wcPi,https://image-cdn-ak.spotifycdn.com/image/ab67...,"(5Qrcxc2YPomtX6UB0bwNaf, Love Is Stronger Than...",Love Is Stronger Than Pride,5Qrcxc2YPomtX6UB0bwNaf
0,837 no pretending,"[2mWfVxEo4xZYDaz0v7hYrN, 1JVEM4C04VPAU9JTnKthN...","[Juna, 23, Love Is Stronger Than Pride, Baby, ...",4gmy8uL4ivI8J8FgZ2wcPi,https://image-cdn-ak.spotifycdn.com/image/ab67...,"(397fxHew7NyjryGWGr1Imy, Baby, This Love I Have)","Baby, This Love I Have",397fxHew7NyjryGWGr1Imy
0,837 no pretending,"[2mWfVxEo4xZYDaz0v7hYrN, 1JVEM4C04VPAU9JTnKthN...","[Juna, 23, Love Is Stronger Than Pride, Baby, ...",4gmy8uL4ivI8J8FgZ2wcPi,https://image-cdn-ak.spotifycdn.com/image/ab67...,"(7Kt7c9Q2B7hSyiTZvGQ37D, Minetta Creek)",Minetta Creek,7Kt7c9Q2B7hSyiTZvGQ37D


In [80]:
songs.reset_index()

,index,name,track_ids,track_names,id,image,zipp,track_name,track_id
0,0,837 no pretending,"[2mWfVxEo4xZYDaz0v7hYrN, 1JVEM4C04VPAU9JTnKthN...","[Juna, 23, Love Is Stronger Than Pride, Baby, ...",4gmy8uL4ivI8J8FgZ2wcPi,https://image-cdn-ak.spotifycdn.com/image/ab67...,"(2mWfVxEo4xZYDaz0v7hYrN, Juna)",Juna,2mWfVxEo4xZYDaz0v7hYrN
1,0,837 no pretending,"[2mWfVxEo4xZYDaz0v7hYrN, 1JVEM4C04VPAU9JTnKthN...","[Juna, 23, Love Is Stronger Than Pride, Baby, ...",4gmy8uL4ivI8J8FgZ2wcPi,https://image-cdn-ak.spotifycdn.com/image/ab67...,"(1JVEM4C04VPAU9JTnKthNk, 23)",23,1JVEM4C04VPAU9JTnKthNk
2,0,837 no pretending,"[2mWfVxEo4xZYDaz0v7hYrN, 1JVEM4C04VPAU9JTnKthN...","[Juna, 23, Love Is Stronger Than Pride, Baby, ...",4gmy8uL4ivI8J8FgZ2wcPi,https://image-cdn-ak.spotifycdn.com/image/ab67...,"(5Qrcxc2YPomtX6UB0bwNaf, Love Is Stronger Than...",Love Is Stronger Than Pride,5Qrcxc2YPomtX6UB0bwNaf
3,0,837 no pretending,"[2mWfVxEo4xZYDaz0v7hYrN, 1JVEM4C04VPAU9JTnKthN...","[Juna, 23, Love Is Stronger Than Pride, Baby, ...",4gmy8uL4ivI8J8FgZ2wcPi,https://image-cdn-ak.spotifycdn.com/image/ab67...,"(397fxHew7NyjryGWGr1Imy, Baby, This Love I Have)","Baby, This Love I Have",397fxHew7NyjryGWGr1Imy
4,0,837 no pretending,"[2mWfVxEo4xZYDaz0v7hYrN, 1JVEM4C04VPAU9JTnKthN...","[Juna, 23, Love Is Stronger Than Pride, Baby, ...",4gmy8uL4ivI8J8FgZ2wcPi,https://image-cdn-ak.spotifycdn.com/image/ab67...,"(7Kt7c9Q2B7hSyiTZvGQ37D, Minetta Creek)",Minetta Creek,7Kt7c9Q2B7hSyiTZvGQ37D
...,...,...,...,...,...,...,...,...,...
9723,924,004 my light in the dawn // searching for a home,"[02VdHwnGhy5fTW2CTZkYsb, 58nPDufBVhMa2bT8G59Cz...","[Helplessness Blues, Via Chicago, Wintersong, ...",3tmZz5OC6gIeGNnxvDjOcZ,https://mosaic.scdn.co/640/ab67616d00001e0221c...,"(3xmra5gBnWr0U4DK3v57F9, I Heard Love Is Blind...",I Heard Love Is Blind - Demo,3xmra5gBnWr0U4DK3v57F9
9724,924,004 my light in the dawn // searching for a home,"[02VdHwnGhy5fTW2CTZkYsb, 58nPDufBVhMa2bT8G59Cz...","[Helplessness Blues, Via Chicago, Wintersong, ...",3tmZz5OC6gIeGNnxvDjOcZ,https://mosaic.scdn.co/640/ab67616d00001e0221c...,"(165cwz4wGlGz0uDBhxdKLY, Right)",Right,165cwz4wGlGz0uDBhxdKLY
9725,924,004 my light in the dawn // searching for a home,"[02VdHwnGhy5fTW2CTZkYsb, 58nPDufBVhMa2bT8G59Cz...","[Helplessness Blues, Via Chicago, Wintersong, ...",3tmZz5OC6gIeGNnxvDjOcZ,https://mosaic.scdn.co/640/ab67616d00001e0221c...,"(2ygvZOXrIeVL4xZmAWJT2C, my future)",my future,2ygvZOXrIeVL4xZmAWJT2C
9726,924,004 my light in the dawn // searching for a home,"[02VdHwnGhy5fTW2CTZkYsb, 58nPDufBVhMa2bT8G59Cz...","[Helplessness Blues, Via Chicago, Wintersong, ...",3tmZz5OC6gIeGNnxvDjOcZ,https://mosaic.scdn.co/640/ab67616d00001e0221c...,"(0HBc7nSv7tT4KXktKAaNYx, September Song)",September Song,0HBc7nSv7tT4KXktKAaNYx


### Audio Features

In [81]:
test = songs.head(50)

In [84]:
audio_features_results = test['track_id'].apply(sp.audio_features)
audio_features_results

0    [{'danceability': 0.606, 'energy': 0.714, 'key...
0    [{'danceability': 0.618, 'energy': 0.366, 'key...
0    [{'danceability': 0.529, 'energy': 0.335, 'key...
0    [{'danceability': 0.59, 'energy': 0.438, 'key'...
0    [{'danceability': 0.761, 'energy': 0.556, 'key...
0    [{'danceability': 0.638, 'energy': 0.413, 'key...
0    [{'danceability': 0.698, 'energy': 0.722, 'key...
0    [{'danceability': 0.391, 'energy': 0.229, 'key...
0    [{'danceability': 0.32, 'energy': 0.479, 'key'...
0    [{'danceability': 0.473, 'energy': 0.359, 'key...
1    [{'danceability': 0.804, 'energy': 0.957, 'key...
1    [{'danceability': 0.756, 'energy': 0.783, 'key...
1    [{'danceability': 0.341, 'energy': 0.845, 'key...
1    [{'danceability': 0.484, 'energy': 0.574, 'key...
1    [{'danceability': 0.754, 'energy': 0.82, 'key'...
1    [{'danceability': 0.553, 'energy': 0.653, 'key...
1    [{'danceability': 0.611, 'energy': 0.863, 'key...
1    [{'danceability': 0.732, 'energy': 0.642, 'key...
2    [{'da

In [86]:
features_db = pd.DataFrame([result[0] for result in audio_features_results])
features_db.head()

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature
0,0.606,0.714,3,-6.244,1,0.0339,0.238,0.098000,0.1060,0.448,141.512,audio_features,2mWfVxEo4xZYDaz0v7hYrN,spotify:track:2mWfVxEo4xZYDaz0v7hYrN,https://api.spotify.com/v1/tracks/2mWfVxEo4xZY...,https://api.spotify.com/v1/audio-analysis/2mWf...,195110,4
1,0.618,0.366,2,-13.760,1,0.0489,0.664,0.682000,0.1280,0.257,88.873,audio_features,1JVEM4C04VPAU9JTnKthNk,spotify:track:1JVEM4C04VPAU9JTnKthNk,https://api.spotify.com/v1/tracks/1JVEM4C04VPA...,https://api.spotify.com/v1/audio-analysis/1JVE...,306187,4
2,0.529,0.335,9,-12.424,1,0.0292,0.766,0.000366,0.0824,0.316,87.577,audio_features,5Qrcxc2YPomtX6UB0bwNaf,spotify:track:5Qrcxc2YPomtX6UB0bwNaf,https://api.spotify.com/v1/tracks/5Qrcxc2YPomt...,https://api.spotify.com/v1/audio-analysis/5Qrc...,258027,4
3,0.590,0.438,6,-12.594,0,0.0508,0.292,0.002160,0.1030,0.727,83.506,audio_features,397fxHew7NyjryGWGr1Imy,spotify:track:397fxHew7NyjryGWGr1Imy,https://api.spotify.com/v1/tracks/397fxHew7Nyj...,https://api.spotify.com/v1/audio-analysis/397f...,250867,4
4,0.761,0.556,6,-9.999,1,0.0483,0.104,0.912000,0.1050,0.905,129.578,audio_features,7Kt7c9Q2B7hSyiTZvGQ37D,spotify:track:7Kt7c9Q2B7hSyiTZvGQ37D,https://api.spotify.com/v1/tracks/7Kt7c9Q2B7hS...,https://api.spotify.com/v1/audio-analysis/7Kt7...,118051,4


In [96]:
features = test.merge(features_db,  left_on='track_id', right_on='id')
features.head()

,name,track_ids,track_names,id_x,image,zipp,track_name,track_id,danceability,energy,...,liveness,valence,tempo,type,id_y,uri,track_href,analysis_url,duration_ms,time_signature
0,837 no pretending,"[2mWfVxEo4xZYDaz0v7hYrN, 1JVEM4C04VPAU9JTnKthN...","[Juna, 23, Love Is Stronger Than Pride, Baby, ...",4gmy8uL4ivI8J8FgZ2wcPi,https://image-cdn-ak.spotifycdn.com/image/ab67...,"(2mWfVxEo4xZYDaz0v7hYrN, Juna)",Juna,2mWfVxEo4xZYDaz0v7hYrN,0.606,0.714,...,0.1060,0.448,141.512,audio_features,2mWfVxEo4xZYDaz0v7hYrN,spotify:track:2mWfVxEo4xZYDaz0v7hYrN,https://api.spotify.com/v1/tracks/2mWfVxEo4xZY...,https://api.spotify.com/v1/audio-analysis/2mWf...,195110,4
1,837 no pretending,"[2mWfVxEo4xZYDaz0v7hYrN, 1JVEM4C04VPAU9JTnKthN...","[Juna, 23, Love Is Stronger Than Pride, Baby, ...",4gmy8uL4ivI8J8FgZ2wcPi,https://image-cdn-ak.spotifycdn.com/image/ab67...,"(1JVEM4C04VPAU9JTnKthNk, 23)",23,1JVEM4C04VPAU9JTnKthNk,0.618,0.366,...,0.1280,0.257,88.873,audio_features,1JVEM4C04VPAU9JTnKthNk,spotify:track:1JVEM4C04VPAU9JTnKthNk,https://api.spotify.com/v1/tracks/1JVEM4C04VPA...,https://api.spotify.com/v1/audio-analysis/1JVE...,306187,4
2,837 no pretending,"[2mWfVxEo4xZYDaz0v7hYrN, 1JVEM4C04VPAU9JTnKthN...","[Juna, 23, Love Is Stronger Than Pride, Baby, ...",4gmy8uL4ivI8J8FgZ2wcPi,https://image-cdn-ak.spotifycdn.com/image/ab67...,"(5Qrcxc2YPomtX6UB0bwNaf, Love Is Stronger Than...",Love Is Stronger Than Pride,5Qrcxc2YPomtX6UB0bwNaf,0.529,0.335,...,0.0824,0.316,87.577,audio_features,5Qrcxc2YPomtX6UB0bwNaf,spotify:track:5Qrcxc2YPomtX6UB0bwNaf,https://api.spotify.com/v1/tracks/5Qrcxc2YPomt...,https://api.spotify.com/v1/audio-analysis/5Qrc...,258027,4
3,837 no pretending,"[2mWfVxEo4xZYDaz0v7hYrN, 1JVEM4C04VPAU9JTnKthN...","[Juna, 23, Love Is Stronger Than Pride, Baby, ...",4gmy8uL4ivI8J8FgZ2wcPi,https://image-cdn-ak.spotifycdn.com/image/ab67...,"(397fxHew7NyjryGWGr1Imy, Baby, This Love I Have)","Baby, This Love I Have",397fxHew7NyjryGWGr1Imy,0.590,0.438,...,0.1030,0.727,83.506,audio_features,397fxHew7NyjryGWGr1Imy,spotify:track:397fxHew7NyjryGWGr1Imy,https://api.spotify.com/v1/tracks/397fxHew7Nyj...,https://api.spotify.com/v1/audio-analysis/397f...,250867,4
4,837 no pretending,"[2mWfVxEo4xZYDaz0v7hYrN, 1JVEM4C04VPAU9JTnKthN...","[Juna, 23, Love Is Stronger Than Pride, Baby, ...",4gmy8uL4ivI8J8FgZ2wcPi,https://image-cdn-ak.spotifycdn.com/image/ab67...,"(7Kt7c9Q2B7hSyiTZvGQ37D, Minetta Creek)",Minetta Creek,7Kt7c9Q2B7hSyiTZvGQ37D,0.761,0.556,...,0.1050,0.905,129.578,audio_features,7Kt7c9Q2B7hSyiTZvGQ37D,spotify:track:7Kt7c9Q2B7hSyiTZvGQ37D,https://api.spotify.com/v1/tracks/7Kt7c9Q2B7hS...,https://api.spotify.com/v1/audio-analysis/7Kt7...,118051,4


In [98]:
playlist_features = features[['name', 'danceability', 'energy', 'key', 'loudness', 'mode', 'speechiness', 'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo']].groupby('name').mean().reset_index()
playlist_features.head()

,name,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo
0,834 heartbreak,0.660500,0.609000,6.250000,-8.015500,1.0,0.044250,0.329625,0.007367,0.113650,0.7460,129.587000
1,835 maybe tomorrow,0.584600,0.495000,4.900000,-8.551300,0.7,0.075680,0.480076,0.182585,0.093510,0.3943,108.079900
2,836 coming around,0.595400,0.678000,6.100000,-7.586400,0.6,0.054480,0.341220,0.297562,0.150880,0.4599,121.993800
3,836 i thought it was too late,0.497500,0.561500,1.500000,-9.450000,0.5,0.032250,0.297500,0.114090,0.114500,0.4345,132.151000
4,836 it falls apart,0.463667,0.578667,6.333333,-8.505833,1.0,0.029533,0.092283,0.122575,0.244833,0.4350,122.427667


In [101]:
playlists[playlists['track_ids']]

,name,track_ids,track_names,id,image,zipp,track_name,track_id
0,837 no pretending,"[2mWfVxEo4xZYDaz0v7hYrN, 1JVEM4C04VPAU9JTnKthN...","[Juna, 23, Love Is Stronger Than Pride, Baby, ...",4gmy8uL4ivI8J8FgZ2wcPi,https://image-cdn-ak.spotifycdn.com/image/ab67...,"(2mWfVxEo4xZYDaz0v7hYrN, Juna)",Juna,2mWfVxEo4xZYDaz0v7hYrN
0,837 no pretending,"[2mWfVxEo4xZYDaz0v7hYrN, 1JVEM4C04VPAU9JTnKthN...","[Juna, 23, Love Is Stronger Than Pride, Baby, ...",4gmy8uL4ivI8J8FgZ2wcPi,https://image-cdn-ak.spotifycdn.com/image/ab67...,"(1JVEM4C04VPAU9JTnKthNk, 23)",23,1JVEM4C04VPAU9JTnKthNk
0,837 no pretending,"[2mWfVxEo4xZYDaz0v7hYrN, 1JVEM4C04VPAU9JTnKthN...","[Juna, 23, Love Is Stronger Than Pride, Baby, ...",4gmy8uL4ivI8J8FgZ2wcPi,https://image-cdn-ak.spotifycdn.com/image/ab67...,"(5Qrcxc2YPomtX6UB0bwNaf, Love Is Stronger Than...",Love Is Stronger Than Pride,5Qrcxc2YPomtX6UB0bwNaf
0,837 no pretending,"[2mWfVxEo4xZYDaz0v7hYrN, 1JVEM4C04VPAU9JTnKthN...","[Juna, 23, Love Is Stronger Than Pride, Baby, ...",4gmy8uL4ivI8J8FgZ2wcPi,https://image-cdn-ak.spotifycdn.com/image/ab67...,"(397fxHew7NyjryGWGr1Imy, Baby, This Love I Have)","Baby, This Love I Have",397fxHew7NyjryGWGr1Imy
0,837 no pretending,"[2mWfVxEo4xZYDaz0v7hYrN, 1JVEM4C04VPAU9JTnKthN...","[Juna, 23, Love Is Stronger Than Pride, Baby, ...",4gmy8uL4ivI8J8FgZ2wcPi,https://image-cdn-ak.spotifycdn.com/image/ab67...,"(7Kt7c9Q2B7hSyiTZvGQ37D, Minetta Creek)",Minetta Creek,7Kt7c9Q2B7hSyiTZvGQ37D
0,837 no pretending,"[2mWfVxEo4xZYDaz0v7hYrN, 1JVEM4C04VPAU9JTnKthN...","[Juna, 23, Love Is Stronger Than Pride, Baby, ...",4gmy8uL4ivI8J8FgZ2wcPi,https://image-cdn-ak.spotifycdn.com/image/ab67...,"(6AcJTLIgNpxs5ERpo03QQc, Serenade of Water)",Serenade of Water,6AcJTLIgNpxs5ERpo03QQc
0,837 no pretending,"[2mWfVxEo4xZYDaz0v7hYrN, 1JVEM4C04VPAU9JTnKthN...","[Juna, 23, Love Is Stronger Than Pride, Baby, ...",4gmy8uL4ivI8J8FgZ2wcPi,https://image-cdn-ak.spotifycdn.com/image/ab67...,"(45JLSLXRMQJabYtPtSuBmV, Guess Who)",Guess Who,45JLSLXRMQJabYtPtSuBmV
0,837 no pretending,"[2mWfVxEo4xZYDaz0v7hYrN, 1JVEM4C04VPAU9JTnKthN...","[Juna, 23, Love Is Stronger Than Pride, Baby, ...",4gmy8uL4ivI8J8FgZ2wcPi,https://image-cdn-ak.spotifycdn.com/image/ab67...,"(1jLNxsmGsEQAM4CUkHemfS, (There Is) No Greater...",(There Is) No Greater Love,1jLNxsmGsEQAM4CUkHemfS
0,837 no pretending,"[2mWfVxEo4xZYDaz0v7hYrN, 1JVEM4C04VPAU9JTnKthN...","[Juna, 23, Love Is Stronger Than Pride, Baby, ...",4gmy8uL4ivI8J8FgZ2wcPi,https://image-cdn-ak.spotifycdn.com/image/ab67...,"(5TOnnVV8YxB4NnRvavrPHX, You the One Fish in t...",You the One Fish in the Sea,5TOnnVV8YxB4NnRvavrPHX
0,837 no pretending,"[2mWfVxEo4xZYDaz0v7hYrN, 1JVEM4C04VPAU9JTnKthN...","[Juna, 23, Love Is Stronger Than Pride, Baby, ...",4gmy8uL4ivI8J8FgZ2wcPi,https://image-cdn-ak.spotifycdn.com/image/ab67...,"(3qTJrHpLz7yY1SRux2Nf3H, Oceanic Feeling)",Oceanic Feeling,3qTJrHpLz7yY1SRux2Nf3H


In [103]:
recs = songs[sp.recommendations(seed_tracks=['2mWfVxEo4xZYDaz0v7hYrN', '1JVEM4C04VPAU9JTnKthNk', '5Qrcxc2YPomtX6UB0bwNaf', '397fxHew7NyjryGWGr1Imy', '7Kt7c9Q2B7hSyiTZvGQ37D', '6AcJTLIgNpxs5ERpo03QQc', '45JLSLXRMQJabYtPtSuBmV', '1jLNxsmGsEQAM4CUkHemfS', '5TOnnVV8YxB4NnRvavrPHX', '3qTJrHpLz7yY1SRux2Nf3H'])]
recs

HTTP Error for GET to https://api.spotify.com/v1/recommendations with Params: {'limit': 20, 'seed_tracks': '2mWfVxEo4xZYDaz0v7hYrN,1JVEM4C04VPAU9JTnKthNk,5Qrcxc2YPomtX6UB0bwNaf,397fxHew7NyjryGWGr1Imy,7Kt7c9Q2B7hSyiTZvGQ37D,6AcJTLIgNpxs5ERpo03QQc,45JLSLXRMQJabYtPtSuBmV,1jLNxsmGsEQAM4CUkHemfS,5TOnnVV8YxB4NnRvavrPHX,3qTJrHpLz7yY1SRux2Nf3H'} returned 400 due to invalid request


SpotifyException: http status: 400, code:-1 - https://api.spotify.com/v1/recommendations?limit=20&seed_tracks=2mWfVxEo4xZYDaz0v7hYrN%2C1JVEM4C04VPAU9JTnKthNk%2C5Qrcxc2YPomtX6UB0bwNaf%2C397fxHew7NyjryGWGr1Imy%2C7Kt7c9Q2B7hSyiTZvGQ37D%2C6AcJTLIgNpxs5ERpo03QQc%2C45JLSLXRMQJabYtPtSuBmV%2C1jLNxsmGsEQAM4CUkHemfS%2C5TOnnVV8YxB4NnRvavrPHX%2C3qTJrHpLz7yY1SRux2Nf3H:
 invalid request, reason: None